# End-to-end Multi-class Dog Breed Classification

This notebook builds multiclass classifier using TensorFlow and TensorFlow Hub.

## 1. Problem 

Identifying the breed of a dog given an image of a dog.

## 2. Data 

The data we are using is from Kaggle's dog breed identification competition.

https://www.kaggle.com/competitions/dog-breed-identification/data

## 3. Evaluation 

The evaluation is a file with prediction probabilities for each dog breed of each test image.

https://www.kaggle.com/competitions/dog-breed-identification/overview/evaluation

## 4. Features

Some information about the data: 
* We are dealing with images (unstructured data) so it is probably best we use deep learning/transfer learning.
* There are 120 breeds of dogs (this means there are 120 different classes).
* There are around 10,000+ images in the training set. (these images have labels).
* There are around 10,000+ images in the test set (these images have no labels, because we will want to predict them).


In [1]:
# Unzip the uploaded date into Google Drive
# !unzip "/content/drive/MyDrive/Dog Vision/dog-breed-identification.zip" -d "/content/drive/MyDrive/Dog Vision"

### Get our workspace ready

* Import TEnsorFlow 2.x
* Import TensoFlow Hub
* Make sure we are using a GPU

In [1]:
# Import TensorFlow and necessary tools into Colab
import tensorflow as tf
import tensorflow_hub as hub
print("TF version:", tf.__version__)
print("TF version:", hub.__version__)

# Check for GPU availability
print("GPU", "available (YESSSSS!!!!!)" if tf.config.list_physical_devices("GPU") else "not available :(")

TF version: 2.9.2
TF version: 0.12.0
GPU available (YESSSSS!!!!!)
